In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats

import jb_helper_functions_prep
from jb_helper_functions_prep import create_enc

import prep_telco
from prep_telco import prep_telco_df

from sklearn.model_selection import train_test_split

In [2]:
df = prep_telco_df()
df.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,onlinesecurity_enc,onlinebackup_enc,deviceprotection_enc,techsupport_enc,streamingtv_enc,streamingmovies_enc,contract_enc,paperlessbilling_enc,paymentmethod_enc,churn_enc
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,0,2,0,0,0,0,0,1,2,0
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,2,0,2,0,0,0,1,0,3,0
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,2,2,0,0,0,0,0,1,3,1
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,2,0,2,2,0,0,1,0,0,0
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,0,0,0,0,0,0,0,1,2,1


In [3]:
train, test = train_test_split(df, test_size=.3, random_state=123, stratify=df[['churn_enc']])

### A few first thoughts:

### Customers with higher monthly costs will churn more.

### Customers with a longer tenure will churn more and pay more per month.

### Customer with month-to-month contracts will churn more frequently.  They will also pay more per month as we would expect a company to have lower rates for a customer willing to sign a year or two contract.

In [4]:
train.columns

Index(['customerid', 'gender', 'seniorcitizen', 'partner', 'dependents',
       'tenure', 'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod', 'monthlycharges', 'totalcharges', 'churn', 'fullyear',
       'gender_enc', 'partner_enc', 'dependents_enc', 'phoneservice_enc',
       'multiplelines_enc', 'internetservice_enc', 'onlinesecurity_enc',
       'onlinebackup_enc', 'deviceprotection_enc', 'techsupport_enc',
       'streamingtv_enc', 'streamingmovies_enc', 'contract_enc',
       'paperlessbilling_enc', 'paymentmethod_enc', 'churn_enc'],
      dtype='object')

In [5]:
pd.DataFrame((train.groupby(['contract', 'churn'])['monthlycharges'].count())/(train.groupby('contract')['monthlycharges'].count()))

monthlycharges
contract       churn                
Month-to-month No           0.573054
               Yes          0.426946
One year       No           0.888454
               Yes          0.111546
Two year       No           0.973639
               Yes          0.026361

### Looking above, we see that 42.7% of month-to-month customers did churn vs. 11.1% of one-year and 2.6% of two-year contract customers.

In [6]:
pd.DataFrame(train.groupby(['internetservice', 'contract'])['monthlycharges'].quantile(.25))

monthlycharges
internetservice contract                      
DSL             Month-to-month         44.4500
                One year               53.7500
                Two year               59.9625
Fiber optic     Month-to-month         76.5500
                One year               92.3000
                Two year               98.9500
No              Month-to-month         19.6000
                One year               19.7000
                Two year               19.7500

In [7]:
pd.DataFrame(train.groupby(['internetservice', 'contract'])['monthlycharges'].quantile(.5))

monthlycharges
internetservice contract                      
DSL             Month-to-month          50.250
                One year                60.950
                Two year                73.100
Fiber optic     Month-to-month          85.700
                One year                99.900
                Two year               105.625
No              Month-to-month          20.050
                One year                20.100
                Two year                20.350

In [8]:
pd.DataFrame(train.groupby(['internetservice', 'contract'])['monthlycharges'].mean())

monthlycharges
internetservice contract                      
DSL             Month-to-month       50.362718
                One year             61.603050
                Two year             70.098190
Fiber optic     Month-to-month       86.804274
                One year             98.655913
                Two year            104.118412
No              Month-to-month       20.307345
                One year             20.883594
                Two year             21.717123

In [9]:
pd.DataFrame(train.groupby(['internetservice', 'contract'])['monthlycharges'].quantile(.75))

monthlycharges
internetservice contract                      
DSL             Month-to-month         57.9500
                One year               71.5000
                Two year               82.4125
Fiber optic     Month-to-month         95.3000
                One year              105.2500
                Two year              111.1125
No              Month-to-month         20.4000
                One year               20.6500
                Two year               24.5875

### In the four tables above, we see that for DSL and Fiber optic, that m-t-m customers are paying less than one and two year contract customers.  This is different than the original thought.  To try and answer this, is it because customers in the m-t-m contracts have a significantly lower tenure so their rates haven't increased yet?  (Think about what happens each year as you're with an internet company.)

### We will engineer a column for years of tenure and take a look at customers and their rates with that approach.  This will be done in our .py so it's created in the entire dataset, our train and test sets will both include this new feature.

In [10]:
pd.DataFrame(train.groupby(['internetservice', 'contract', 'churn'])['monthlycharges'].mean())

monthlycharges
internetservice contract       churn                
DSL             Month-to-month No          52.006973
                               Yes         46.821245
                One year       No          61.565103
                               Yes         61.962500
                Two year       No          70.147465
                               Yes         67.425000
Fiber optic     Month-to-month No          87.676488
                               Yes         86.071402
                One year       No          97.831761
                               Yes        102.347183
                Two year       No         104.168978
                               Yes        103.488636
No              Month-to-month No          20.327289
                               Yes         20.226429
                One year       No          20.887952
                               Yes         20.728571
                Two year       No          21.721625
                               Yes         19.750000

### I would have thought that customers that churned would be the higher paying customers.  This shows that they are not.

In [15]:
pd.DataFrame(train.groupby(['internetservice', 'contract', 'fullyear'])['monthlycharges'].mean())

monthlycharges
internetservice contract       fullyear                
DSL             Month-to-month 0              47.882096
                               1              52.568132
                               2              53.808036
                               3              53.279464
                               4              51.632432
                               5              59.021875
                One year       0              53.072500
                               1              59.692857
                               2              62.054667
                               3              61.093750
                               4              63.422892
                               5              64.102727
                Two year       0              54.750000
                               1              56.700000
                               2              67.144118
                               3              67.252632
                               4              67.137313
                               5              70.679126
                               6              76.972159
Fiber optic     Month-to-month 0              81.562341
                               1              86.753872
                               2              89.384141
                               3              92.264430
                               4              94.635772
                               5              97.736905
                               6             112.900000
                One year       0              94.433333
                               1              92.255882
                               2              93.611905
                               3              97.523770
                               4              99.692273
                               5             100.311724
                               6             103.045455
                Two year       2              98.860000
                               3              99.617857
                               4              98.843056
                               5             103.984043
                               6             107.100000
No              Month-to-month 0              20.233270
                               1              20.505085
                               2              20.806000
                               3              19.620000
                               4              20.000000
                               5              19.400000
                One year       0              20.364583
                               1              20.461111
                               2              20.890385
                               3              21.343056
                               4              21.132895
                               5              22.207895
                Two year       0              20.737037
                               1              21.460417
                               2              21.468889
                               3              21.112500
                               4              21.395455
                               5              22.134926
                               6              23.097561

### As expected, almost all subgroups show a trend upwards in monthly charges.

In [16]:
pd.DataFrame(train.groupby(['internetservice', 'contract', 'fullyear'])['monthlycharges'].count())

monthlycharges
internetservice contract       fullyear                
DSL             Month-to-month 0                    458
                               1                    182
                               2                    112
                               3                     56
                               4                     37
                               5                     16
                One year       0                     20
                               1                     56
                               2                     75
                               3                     88
                               4                     83
                               5                     55
                Two year       0                     10
                               1                     16
                               2                     17
                               3                     38
                               4                     67
                               5                    206
                               6                     88
Fiber optic     Month-to-month 0                    628
                               1                    297
                               2                    227
                               3                    149
                               4                    123
                               5                     84
                               6                      1
                One year       0                      3
                               1                     17
                               2                     42
                               3                     61
                               4                    110
                               5                    145
                               6                     11
                Two year       2                      5
                               3                     14
                               4                     36
                               5                    141
                               6                    100
No              Month-to-month 0                    263
                               1                     59
                               2                     25
                               3                      5
                               4                      1
                               5                      1
                One year       0                     48
                               1                     63
                               2                     52
                               3                     36
                               4                     38
                               5                     19
                Two year       0                     27
                               1                     48
                               2                     45
                               3                     64
                               4                     77
                               5                    136
                               6                     41

In [17]:
df.columns

Index(['customerid', 'gender', 'seniorcitizen', 'partner', 'dependents',
       'tenure', 'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod', 'monthlycharges', 'totalcharges', 'churn', 'fullyear',
       'gender_enc', 'partner_enc', 'dependents_enc', 'phoneservice_enc',
       'multiplelines_enc', 'internetservice_enc', 'onlinesecurity_enc',
       'onlinebackup_enc', 'deviceprotection_enc', 'techsupport_enc',
       'streamingtv_enc', 'streamingmovies_enc', 'contract_enc',
       'paperlessbilling_enc', 'paymentmethod_enc', 'churn_enc'],
      dtype='object')

In [19]:
pd.DataFrame(train.groupby(['internetservice', 'contract', 'churn'])['churn'].count())

churn
internetservice contract       churn       
DSL             Month-to-month No       588
                               Yes      273
                One year       No       341
                               Yes       36
                Two year       No       434
                               Yes        8
Fiber optic     Month-to-month No       689
                               Yes      820
                One year       No       318
                               Yes       71
                Two year       No       274
                               Yes       22
No              Month-to-month No       284
                               Yes       70
                One year       No       249
                               Yes        7
                Two year       No       437
                               Yes        1

### Looking at the table above, we see that over half of fiber optic m-t-m churned.  DSL m-t-m had a high churn rate as well.

In [34]:
list(df.select_dtypes(include=['object']).columns)

['customerid',
 'gender',
 'partner',
 'dependents',
 'phoneservice',
 'multiplelines',
 'internetservice',
 'onlinesecurity',
 'onlinebackup',
 'deviceprotection',
 'techsupport',
 'streamingtv',
 'streamingmovies',
 'contract',
 'paperlessbilling',
 'paymentmethod',
 'churn']

In [56]:
list(df.columns[df.nunique()<5])

['gender',
 'seniorcitizen',
 'partner',
 'dependents',
 'phoneservice',
 'multiplelines',
 'internetservice',
 'onlinesecurity',
 'onlinebackup',
 'deviceprotection',
 'techsupport',
 'streamingtv',
 'streamingmovies',
 'contract',
 'paperlessbilling',
 'paymentmethod',
 'churn',
 'gender_enc',
 'partner_enc',
 'dependents_enc',
 'phoneservice_enc',
 'multiplelines_enc',
 'internetservice_enc',
 'onlinesecurity_enc',
 'onlinebackup_enc',
 'deviceprotection_enc',
 'techsupport_enc',
 'streamingtv_enc',
 'streamingmovies_enc',
 'contract_enc',
 'paperlessbilling_enc',
 'paymentmethod_enc',
 'churn_enc']

In [60]:
df[(df.columns[df.nunique()<5]) & (df.select_dtypes(include=['object']))].columns

Index([], dtype='object')

In [64]:
list(df[df.columns[df.nunique()<5]].select_dtypes(include=['object']).columns )

['gender',
 'partner',
 'dependents',
 'phoneservice',
 'multiplelines',
 'internetservice',
 'onlinesecurity',
 'onlinebackup',
 'deviceprotection',
 'techsupport',
 'streamingtv',
 'streamingmovies',
 'contract',
 'paperlessbilling',
 'paymentmethod',
 'churn']